In [1]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, export_png
from IPython.core.display import HTML
import pandas as pd

In [2]:
style_table = open('/Users/Scott/Desktop/Data/style-table.css').read()
style_notebook = open('/Users/Scott/Desktop/Data/style-notebook.css').read()
HTML('<style>{}</style>'.format(style_table + style_notebook))

## ISEE Trends by Year
#### This notebook takes 14 years of admissions data and generates graphs to visualize standardized testing trends over those years.
*Data has been altered for the purposes of confidentitality.

In [3]:
# Takes admissions Excel files by year, converts them into pandas dataframes, and stores them in 
# a list called files.

counter = 4
files = []

for _ in range(5):
    year = str(counter)+'_0'+str(counter+1)
    files.append('/Users/Scott/Desktop/Data/Admissions/original_files/0'+year+'.xlsx')
    counter += 1
    
for _ in range(1):
    year = str(counter)+'_'+str(counter+1)
    files.append('/Users/Scott/Desktop/Data/Admissions/original_files/0'+year+'.xlsx')
    counter += 1
    
for _ in range(7):
    year = str(counter)+'_'+str(counter+1)
    files.append('/Users/Scott/Desktop/Data/Admissions/original_files/'+year+'.xlsx')
    counter += 1
    
for i, file in enumerate(files):
    files[i] = pd.read_excel(file)
    files[i] = files[i].dropna(axis=1, how="all")
    files[i] = files[i].dropna(axis=0, how="all")

## Graphs of incoming Math, Quantitative, Verbal, Reading ISEE scores by year for 6th grade applicants, 9th, and all new students

##### Creates dictionary of standardized testing scores by year admitted and subject

In [4]:
# Selects admitted students and puts them into dataframes for analysis, by year and subject
# Accounts for the different years in file names

dfs_subj_and_grade = {}
subject_col_names = ['ISEE_Math', 'ISEE_Quantitative', 'ISEE_Verbal', 'ISEE_Reading']
grades = {'sixth': 6, 'ninth': 9}

for subj in subject_col_names:
    for grade in grades:
        # starts with 2004, then joins other years
        counter = 4 
        students_by_year = files[0][files[0].Grade_apply==grades[grade]]
        by_subj = students_by_year.columns.intersection([subj])
        scores_by_grade_and_subject = students_by_year[by_subj].dropna(how="any")
        scores_by_grade_and_subject.columns = ['200'+str(counter)]
        subj_and_grade = str(subj)+'_'+str(grade)
        
        # create dictionary key of SUBJECT_GRADE ADMITTED, e.g "ISEE_MATH_sixth"
        dfs_subj_and_grade[subj_and_grade] = scores_by_grade_and_subject
        
        # concat all other years (2005 to 2016) by subject to the dataframe
        for i in range(len(files)-1):
            counter += 1
            students_by_year = files[i+1][files[i+1].Grade_apply==grades[grade]]

            if counter <= 9:
                new_year = students_by_year[by_subj].dropna(how="any")
                new_year.columns = ['200'+str(counter)]

            else:
                new_year = students_by_year[by_subj].dropna(how="any")
                new_year.columns = ['20'+str(counter)]
            
            dfs_subj_and_grade[subj_and_grade]= pd.concat([dfs_subj_and_grade[subj_and_grade],
                                                              new_year], axis=1)

In [5]:
print(subj_and_grade)
dfs_subj_and_grade[subj_and_grade].describe().drop(['mean', 'std','min','max'])


ISEE_Reading_ninth


,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
count,21.0,16.0,27.0,19.0,27.0,33.0,20.00,24.00,21.0,27.0,22.0,9.0,14.0
25%,51.0,43.5,56.0,68.5,71.0,56.0,72.50,74.75,65.0,66.0,87.0,87.0,66.5
50%,66.0,73.0,79.0,85.0,85.0,74.0,87.00,87.00,77.0,83.0,91.0,89.0,91.0
75%,85.0,88.5,92.5,99.0,93.0,92.0,87.75,90.25,90.0,93.0,95.0,93.0,93.0


In [6]:
# edits key names for chart titles for aesthetics
for old_key in dfs_subj_and_grade:
    try:
        key_list = old_key.split('_')
        new_key = key_list[2].capitalize()+' Grade '+key_list[1].capitalize()+' Admissions Scores'
        dfs_subj_and_grade[new_key] = dfs_subj_and_grade.pop(old_key)
    except:
        pass

## Outputs graphs showing trends of the 75%, 50%, and 25% standardized test scores by subject and class

##### Gives admissions and administration a sense of whether our students have grown academically stronger or weaker through the years

In [7]:
# Outputs 8 graphs of 6th and 9th grade admissions scores for the four subjects

for graph_title in dfs_subj_and_grade:
    output_notebook()
    
    graph = dfs_subj_and_grade[graph_title].describe().drop(['mean', 'std','min','max'])

    p = figure(title=graph_title, plot_width=400, plot_height=400)
    p.xaxis.axis_label = 'Year Admitted'
    p.yaxis.axis_label = 'Percentile'

    leg = ["75%","50%","25%"]
    col = ["green", "navy", "firebrick"]
    counter = 0
    years = [2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]
    for i in [graph.values.tolist()[3], 
              graph.values.tolist()[2], 
              graph.values.tolist()[1]]:
        p.line(x=years, y=i, legend = leg[counter], color = col[counter])
        counter += 1

    p.legend.location = "bottom_right"

    export_png(p, filename="results/"+graph_title+".png")

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...